In [1]:
!pip install -qq datasets
!pip install -qq transformers

     |████████████████████████████████| 365 kB 34.7 MB/s 
     |████████████████████████████████| 141 kB 61.7 MB/s 
     |████████████████████████████████| 115 kB 73.2 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 127 kB 71.5 MB/s 
     |████████████████████████████████| 4.7 MB 36.4 MB/s 
     |████████████████████████████████| 6.6 MB 17.4 MB/s 


### Preparing The dataset

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset('conll2003')

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Inspecting the data
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
# Looking at first element in the dataset
raw_datasets['train']['tokens'][0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [5]:
# look at ner tags for named-entity-recognition
raw_datasets['train']['ner_tags'][0]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [6]:
# inspect feature attribute to see mappings of tags to category
ner_features = raw_datasets['train'].features['ner_tags']
ner_features

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
label_names = ner_features.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
# decoding labels
words = raw_datasets['train']['tokens'][0]
labels = raw_datasets['train']['ner_tags'][0]
line1 = ''
line2 = ''

for word, label in zip(words, labels):
  full_label = label_names[label]
  max_length = max(len(word), len(full_label))
  line1 += word + " " * (max_length - len(word) + 1)
  line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


#### Excerise 1:

print train examples 1 and 4 but this time with thier pos tags and chunks

In [9]:
pos_features = raw_datasets['train'].features['pos_tags']
chunk_features = raw_datasets['train'].features['chunk_tags']

pos_label_names = pos_features.feature.names
chunk_label_names = chunk_features.feature.names

word1 = raw_datasets['train']['tokens'][0]
word4 = raw_datasets['train']['tokens'][4]

pos_label1 = raw_datasets['train']['pos_tags'][0]
pos_label4 = raw_datasets['train']['pos_tags'][4]
chunk_label1 = raw_datasets['train']['chunk_tags'][0]
chunk_label4 = raw_datasets['train']['chunk_tags'][4]

def decode_labels(words, labels, label_names):
  '''Function decodes labels per word and prints them '''
  line1 = ''
  line2 = ''
  for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + ' ' * (max_length - len(word) + 1)
    line2 += full_label + ' ' * (max_length - len(full_label) + 1)

  print(line1)
  print(line2)

In [10]:
# decode pos tags for word 1 and 4
decode_labels(word1,pos_label1, pos_label_names)
decode_labels(word4,pos_label4, pos_label_names)  

EU  rejects German call to boycott British lamb . 
NNP VBZ     JJ     NN   TO VB      JJ      NN   . 
Germany 's  representative to the European Union 's  veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
NNP     POS NN             TO DT  NNP      NNP   POS JJ         NN        NNP    NNP       VBD  IN NNP       NNS       MD     VB  NN        IN   NNS       JJ    IN   NNP     IN    DT  JJ         NN     VBD JJR     . 


In [11]:
# decode chunk tags for word 1 and 4
decode_labels(word1,chunk_label1, chunk_label_names)
decode_labels(word4,chunk_label4, chunk_label_names)  

EU   rejects German call to   boycott British lamb . 
B-NP B-VP    B-NP   I-NP B-VP I-VP    B-NP    I-NP O 
Germany 's   representative to   the  European Union 's   veterinary committee Werner Zwingmann said on   Wednesday consumers should buy  sheepmeat from countries other  than Britain until  the  scientific advice was  clearer . 
B-NP    B-NP I-NP           B-PP B-NP I-NP     I-NP  B-NP I-NP       I-NP      I-NP   I-NP      B-VP B-PP B-NP      I-NP      B-VP   I-VP B-NP      B-PP B-NP      B-ADJP B-PP B-NP    B-SBAR B-NP I-NP       I-NP   B-VP B-ADJP  O 


### Data Processing

In [12]:
from transformers import AutoTokenizer

In [13]:
model_checkpoint = 'bert-base-cased' # initialize a checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
# test tokenization
inputs = tokenizer(raw_datasets['train']['tokens'][0], is_split_into_words=True) # flag when inputs are pre-tokenized
print(inputs.tokens())

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


In [15]:
inputs.word_ids()  # for aligning labels

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [16]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
        # Special token
        new_labels.append(-100)
    else:
        # Same word as previous token
        label = labels[word_id]
        # If the label is B-XXX we change it to I-XXX
        if label % 2 == 1:
            label += 1
        new_labels.append(label)

  return new_labels

In [17]:
## Test aligning labels
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


#### Excerise 2:

Some researchers prefer to attribute only one label per word, and assign -100 to the other subtokens in a given word. This is to avoid long words that split into lots of subtokens contributing heavily to the loss. Change the previous function to align labels with input IDs by following this rule.

In [18]:
def align_labels_with_tokens2(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id == None else labels[word_id]
      new_labels.append(label)
    elif word_id == None:
      new_labels.append(-100)
    else:
      new_labels.append(-100)
  return new_labels

In [19]:
# test excerise function
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens2(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, -100, 0, -100]


#### Preprocess all

In [20]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples['tokens'], truncation=True, is_split_into_words=True
  )
  all_labels = examples['ner_tags']
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels
  return tokenized_inputs

In [21]:
# apply all
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched = True,
    remove_columns=raw_datasets["train"].column_names
)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

### Fine-tuning the model with the Trainer API

#### Data Collation

Use DataCollatorForTokenClassification that pads inputs along with labels

In [22]:
from transformers import DataCollatorForTokenClassification

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer= tokenizer)

In [24]:
# test collator
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [25]:
# compare with actual labels
for i in range(2):
    print(tokenized_datasets["train"][i]['labels'])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


#### Metrics

compute metrics using seqeval

In [26]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=2176599affa565ed1c5172c16132993674c3b3702e85a6398ad2deb12b3162c5
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [27]:
!pip install -qq evaluate

     |████████████████████████████████| 69 kB 6.2 MB/s 


In [28]:
import evaluate

In [29]:
# load it via evaluate
metric = evaluate.load('seqeval')

In [30]:
# the metric above takes labels as strings
# test decoding labels
labels = raw_datasets['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [31]:
# create fake prediction
predictions = labels.copy()
predictions[2] = 'O'
metric.compute(predictions= [predictions], references= [labels])

{'MISC': {'f1': 0.6666666666666666,
  'number': 2,
  'precision': 1.0,
  'recall': 0.5},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 0.8888888888888888,
 'overall_f1': 0.8,
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666}

In [47]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

#### Defining the model

In [48]:
# set label mappings
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [49]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps

In [50]:
# test if model has the correct label names
model.config.num_labels

9

#### Fine tuning the model

In [51]:
from huggingface_hub import notebook_login

notebook_login() # log into hub

In [52]:
# define training args
from transformers import TrainingArguments

args = TrainingArguments(
    'bert-finetuned-ner',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [53]:
# define Trainer and train
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5268


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.086700,0.071644,0.910199,0.929653,0.919823,0.981972
2,0.034500,0.067978,0.928973,0.946483,0.937646,0.985386
3,0.019100,0.064143,0.929043,0.947492,0.938177,0.985783


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-1756
Configuration saved in bert-finetuned-ner/checkpoint-1756/config.json
Model weights saved in bert-finetuned-ner/checkpoint-1756/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-1756/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-1756/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-3512
Configuration saved in bert-finetuned-ner/checkpoint-3512/config.json
Model weights saved in bert-finetuned-ner/checkpoint-3512/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-3512/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-3512/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
Saving model checkpoin

TrainOutput(global_step=5268, training_loss=0.06672658798996933, metrics={'train_runtime': 605.9414, 'train_samples_per_second': 69.517, 'train_steps_per_second': 8.694, 'total_flos': 923954298531210.0, 'train_loss': 0.06672658798996933, 'epoch': 3.0})

In [ ]:
# trainer.push_to_hub(commit_message="Training complete")

### A custom training loop

In [54]:
# prepare for training
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle = True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [55]:
#  reinstantiate  model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps

In [56]:
# define optimizers
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [59]:
!pip install -qq accelerate

     |████████████████████████████████| 143 kB 33.1 MB/s 


In [61]:
from accelerate import Accelerator

In [62]:
# prepare accelerator
accelerator = Accelerator()

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [63]:
from transformers import get_scheduler

In [64]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

#### Training Loop

In [66]:
output_dir = "bert-finetuned-ner-accelerate"

In [65]:
# define a post-process function for easy evaluation
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [67]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

  0%|          | 0/5268 [00:00<?, ?it/s]

Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 0: {'precision': 0.929821608885897, 'recall': 0.904995904995905, 'f1': 0.9172408068398771, 'accuracy': 0.9806175310531583}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 1: {'precision': 0.9419387411645911, 'recall': 0.9166393711103832, 'f1': 0.9291168658698539, 'accuracy': 0.9849443692235239}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 2: {'precision': 0.9464826657691013, 'recall': 0.9233295025447381, 'f1': 0.9347627358098564, 'accuracy': 0.9855036204156119}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json


#### Test model with pipeline

In [69]:
from transformers import pipeline

# Using my local pipeline
model_checkpoint = '/content/bert-finetuned-ner/checkpoint-5268'
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy='simple'
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

loading configuration file /content/bert-finetuned-ner/checkpoint-5268/config.json
Model config BertConfig {
  "_name_or_path": "/content/bert-finetuned-ner/checkpoint-5268",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pos

[{'end': 18,
  'entity_group': 'PER',
  'score': 0.999073,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.9650182,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.99601525,
  'start': 49,
  'word': 'Brooklyn'}]